# Word Alignment Tutorial

In this notebook, we will demonstrate how to use machine to train statistical word alignment models and then use them to predict alignments between sentences. Machine uses the [Thot](https://github.com/sillsdev/thot) library to implement word alignment models. The classes can be enabled by installing the `sil-machine` package with the `thot` optional dependency.

## Training models

The first step in training a statistical word alignment model is setting up a parallel text corpus. Word alignment models are unsupervised, so they only require a parallel text corpus to train. Manually created alignments are not necessary. So let's create a `ParallelTextCorpus` instance from the source and target monolingual corpora.

In [6]:
from machine.corpora import ParallelTextCorpus, ParatextTextCorpus

source_corpus = ParatextTextCorpus("data/VBL-PT")
target_corpus = ParatextTextCorpus("data/WEB-PT")
parallel_corpus = ParallelTextCorpus(source_corpus, target_corpus)

Machine has implementations of all common statistical models, including the famous IBM models (1-4), HMM, and FastAlign. All alignment models implement the `WordAlignmentModel` abstract base class. This makes it easier to swap out different models in your code.

In this tutorial, we are going to start by training an IBM-1 model. There are two possible ways to train a model. First, we will demonstrate training a model from a class that inherits from `WordAlignmentModel`. We use the `create_trainer` method to create a trainer object that is used to train the model. If we do not specify a file path when creating the model object, then the model will only exist in memory. When we call the `save` method, the model instance is updated with the trained model parameters, but the model is not written to disk. We need to preprocess the corpus before training. First, we need to tokenize the corpus. We will also lowercase the data, since that generally gives better results.

In [7]:
from machine.translation.thot import ThotIbm1WordAlignmentModel
from machine.tokenization import LatinWordTokenizer

tokenizer = LatinWordTokenizer()
parallel_corpus = parallel_corpus.tokenize(tokenizer).lowercase()

model = ThotIbm1WordAlignmentModel()
trainer = model.create_trainer(parallel_corpus)
trainer.train(lambda status: print(f"Training IBM-1 model: {status.percent_completed:.2%}"))
trainer.save()

Training IBM-1 model: 0.00%
Training IBM-1 model: 16.67%
Training IBM-1 model: 33.33%
Training IBM-1 model: 50.00%
Training IBM-1 model: 66.67%
Training IBM-1 model: 83.33%
Training IBM-1 model: 100.00%


The other option for training a model is to construct a trainer object directly. This method is useful for when you are only interested in training the model and saving it to disk for later use. We need to specify where the model will be saved after it is trained and we call the `save` method.

In [8]:
import os
from machine.translation.thot import ThotWordAlignmentModelTrainer, ThotWordAlignmentModelType

os.makedirs("out/VBL-WEB-IBM1", exist_ok=True)
trainer = ThotWordAlignmentModelTrainer(ThotWordAlignmentModelType.IBM1, parallel_corpus, "out/VBL-WEB-IBM1/src_trg")

trainer.train(lambda status: print(f"Training IBM-1 model: {status.percent_completed:.2%}"))
trainer.save()
print("IBM-1 model saved")

Training IBM-1 model: 0.00%
Training IBM-1 model: 16.67%
Training IBM-1 model: 33.33%
Training IBM-1 model: 50.00%
Training IBM-1 model: 66.67%
Training IBM-1 model: 83.33%
Training IBM-1 model: 100.00%
IBM-1 model saved


## Aligning parallel sentences

Now that we have a trained alignment model, we can find the best alignment for a parallel sentence. We call `get_best_alignment` method to find the best alignment. The results are returned as a `WordAlignmentMatrix` object.

In [9]:
from itertools import islice

model = ThotIbm1WordAlignmentModel("out/VBL-WEB-IBM1/src_trg")
for row in islice(parallel_corpus.get_rows(), 5):
    alignment = model.get_best_alignment(row.source_segment, row.target_segment)

    print(row.ref)
    print("Source:", row.source_text)
    print("Target:", row.target_text)
    print("Alignment:", str(alignment))

1JN 1:1
Source: esta carta trata sobre la palabra de vida que existía desde el principio , que hemos escuchado , que hemos visto con nuestros propios ojos y le hemos contemplado , y que hemos tocado con nuestras manos .
Target: that which was from the beginning , that which we have heard , that which we have seen with our eyes , that which we saw , and our hands touched , concerning the word of life
Alignment: 15-0 16-1 2-2 12-3 4-4 12-5 13-6 15-7 16-8 35-9 15-10 16-11 13-12 15-13 16-14 35-15 15-16 20-17 21-18 22-19 2-20 13-21 15-22 16-23 35-24 2-25 13-26 25-27 22-28 2-29 2-30 13-31 3-32 4-33 5-34 6-35 7-36
1JN 1:2
Source: esta vida nos fue revelada . la vimos y damos testimonio de ella . estamos hablándoles de aquél que es la vida eterna , que estaba con el padre , y que nos fue revelado .
Target: ( and the life was revealed , and we have seen , and testify , and declare to you the life , the eternal life , which was with the father , and was revealed to us ) ;
Alignment: 4-0 8-1 28-2

## Getting model probabilities

A statistical word alignment model consists of one or more conditional probability distributions that are estimated from the training data. For example, most models estimate a word translation probability distribution that can be queried to obtain the probability that a source word is a translation of a target word. Each model class has methods to obtain these probabilities. Let's try getting some translation probabilities from the IBM-1 model that we trained by calling the `get_translation_probability` method. In order to get the probability that a word does not translate to anything, you can pass `None` instead of the word string.

In [10]:
model = ThotIbm1WordAlignmentModel("out/VBL-WEB-IBM1/src_trg")
prob = model.get_translation_probability("es", "is")
print(f"es -> is: {prob:.4f}")
prob = model.get_translation_probability(None, "that")
print(f"NULL -> that: {prob:.4f}")

es -> is: 0.2720
NULL -> that: 0.0516


## Symmetrized alignment models

Most statistical word alignment models are directional and asymmetric. This means that it can only model one-to-one and one-to-many alignments in one direction. They are not capable of modeling many-to-many alignments, which can occur in some language pairs. One way to get around this limitation is to train models in both directions (source-to-target and target-to-source), and then merge the resulting alignments from the two models into a single alignment. This is called symmetrization and is a common practice when using statistical word alignment models. In addition, researchers have found that symmetrized alignments are better quality.

Machine provides a special word alignment model class to support symmetrization called `ThotSymmetrizedWordAlignmentModel`. Let's demonstrate how to use this class. First, we will train the symmetrized model using the `SymmetrizedWordAlignmentModelTrainer` class.

In [11]:
from machine.translation import SymmetrizedWordAlignmentModelTrainer

src_trg_trainer = ThotWordAlignmentModelTrainer(
    ThotWordAlignmentModelType.IBM1, parallel_corpus, "out/VBL-WEB-IBM1/src_trg"
)
trg_src_trainer = ThotWordAlignmentModelTrainer(
    ThotWordAlignmentModelType.IBM1, parallel_corpus.invert(), "out/VBL-WEB-IBM1/trg_src"
)
symmetrized_trainer = SymmetrizedWordAlignmentModelTrainer(src_trg_trainer, trg_src_trainer)
symmetrized_trainer.train(lambda status: print(f"{status.message}: {status.percent_completed:.2%}"))
symmetrized_trainer.save()
print("Symmetrized IBM-1 model saved")

Training direct alignment model: 0.00%
Training direct alignment model: 0.00%
Training direct alignment model: 8.33%
Training direct alignment model: 16.67%
Training direct alignment model: 25.00%
Training direct alignment model: 33.33%
Training direct alignment model: 41.67%
Training direct alignment model: 50.00%
Training inverse alignment model: 50.00%
Training inverse alignment model: 50.00%
Training inverse alignment model: 58.33%
Training inverse alignment model: 66.67%
Training inverse alignment model: 75.00%
Training inverse alignment model: 83.33%
Training inverse alignment model: 91.67%
Training inverse alignment model: 100.00%
Symmetrized IBM-1 model saved


The model can also be trained using the `create_trainer` method on `ThotSymmetrizedWordAlignmentModel`. Now that we've trained the symmetrized model, let's obtain some alignments. Machine supports many different symmetrization heuristics. The symmetrization heuristic to use when merging alignments can be specified using the `heuristic` property. In this case, we will use the `GROW_DIAG_FINAL_AND` heuristic.

In [12]:
from machine.translation import SymmetrizationHeuristic
from machine.translation.thot import ThotSymmetrizedWordAlignmentModel

src_trg_model = ThotIbm1WordAlignmentModel("out/VBL-WEB-IBM1/src_trg")
trg_src_model = ThotIbm1WordAlignmentModel("out/VBL-WEB-IBM1/trg_src")
symmetrized_model = ThotSymmetrizedWordAlignmentModel(src_trg_model, trg_src_model)
symmetrized_model.heuristic = SymmetrizationHeuristic.GROW_DIAG_FINAL_AND
for row in islice(parallel_corpus.get_rows(), 5):
    alignment = model.get_best_alignment(row.source_segment, row.target_segment)

    print(row.ref)
    print("Source:", row.source_text)
    print("Target:", row.target_text)
    print("Alignment:", alignment)

1JN 1:1
Source: esta carta trata sobre la palabra de vida que existía desde el principio , que hemos escuchado , que hemos visto con nuestros propios ojos y le hemos contemplado , y que hemos tocado con nuestras manos .
Target: that which was from the beginning , that which we have heard , that which we have seen with our eyes , that which we saw , and our hands touched , concerning the word of life
Alignment: 15-0 16-1 2-2 12-3 4-4 12-5 13-6 15-7 16-8 35-9 15-10 16-11 13-12 15-13 16-14 35-15 15-16 20-17 21-18 22-19 2-20 13-21 15-22 16-23 35-24 2-25 13-26 25-27 22-28 2-29 2-30 13-31 3-32 4-33 5-34 6-35 7-36
1JN 1:2
Source: esta vida nos fue revelada . la vimos y damos testimonio de ella . estamos hablándoles de aquél que es la vida eterna , que estaba con el padre , y que nos fue revelado .
Target: ( and the life was revealed , and we have seen , and testify , and declare to you the life , the eternal life , which was with the father , and was revealed to us ) ;
Alignment: 4-0 8-1 28-2